# INVESTIGANDO A POBREZA NO BRASIL

Para investigar como a pobreza no brasil se comporta vamos analisar primeiramente como cada gênero e raca é afetado pela pobreza ou vulnerabilidade

### Hipótese 1: A pobreza afeta mais as mulheres do que os homens. 
    Esta hipótese sugere que as mulheres são desproporcionalmente afetadas pela pobreza em comparação aos homens, possivelmente devido a fatores como discriminação de gênero, desigualdade salarial e responsabilidades familiares.

### Hipótese 2: A pobreza afeta mais as pessoas de raça negra do que as de raça branca. 
    Esta hipótese propõe que as pessoas de raça negra são mais afetadas pela pobreza do que as de raça branca, possivelmente devido a fatores históricos e estruturais de desigualdade racial.

### Hipótese 3: A interseção de gênero e raça tem um impacto significativo na pobreza. 
    Esta hipótese sugere que a combinação de gênero e raça pode ter um impacto significativo na experiência da pobreza. Por exemplo, uma mulher negra pode enfrentar desafios únicos que são diferentes dos enfrentados por um homem branco ou uma mulher branca.

### Hipótese 4: A disparidade na pobreza entre diferentes grupos raciais é maior entre as mulheres do que entre os homens. 
    Esta hipótese sugere que a diferença na experiência da pobreza entre diferentes grupos raciais é mais pronunciada entre as mulheres do que entre os homens. Por exemplo, a diferença na taxa de pobreza entre mulheres negras e mulheres brancas pode ser maior do que a diferença entre homens negros e homens brancos.

Para iniciar nossa analise vamos iniciar nossa base de dados "UF_AgregadoporSexoRaca_20230419.csv"

Além disso vamos verificar inconsistências no databse 

In [60]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import matplotlib.pyplot as plt
import seaborn as sns

# import warnings
# warnings.filterwarnings('ignore')



dataframeBase = pd.read_csv("./Agregações sexo e raça/UF_AgregadoporSexoRaca_20230419.csv", sep=",")

zeros_por_coluna = dataframeBase.apply(lambda coluna: (coluna == 0).sum())
colunas = dataframeBase.columns.tolist()

for i,coluna in enumerate(zeros_por_coluna):
    if coluna != 0 and colunas[i] != "local":
        print(f"Coluna {colunas[i]} tem {coluna} zeros")

tmpDF = dataframeBase[["qtd_pes_pobres", "qtd_pes_vulneraveis", "qtd_pes_pob_vul","qtd_0a11","qtd_pessoas"]]

colunas = tmpDF.columns.tolist()
        
zeros_por_coluna = tmpDF.apply(lambda coluna: (coluna > tmpDF['qtd_pessoas']).sum())
for i,coluna in enumerate(zeros_por_coluna):
    if coluna != 0 and colunas[i] != "local":
        print(f"Coluna {colunas[i]} tem {coluna} inconsistencias")


Coluna qtd_pes_pobres tem 116 inconsistencias
Coluna qtd_pes_pob_vul tem 123 inconsistencias


### Checamos a quantidade de infos vazias na nossa database e chegamos a conclusão que todos os dados estão completos! mas ...
    Verificamos a quantidade de 0 indesejados no nosso database e algumas inconsistências em que o numero de pessoas pobres ou em vulnerabilidade é maior do que o numero de pessoas, entretanto nossos dados do Brasil inteiro (linha 1) estão completos, então vamos utilizar eles para uma primeira analise
#### Agora vamos selecionar os dados que queremos comparar em um novo database, sao eles apenas:
    sxraca
    qtd_pessoas
    qtd_pes_pobres
    qtd_pes_vulneraveis
    qtd_pes_pob_vul
    qtd_0a11

In [61]:
newDataframe = dataframeBase[["local","sxraca", "qtd_pessoas", "qtd_pes_pobres", "qtd_pes_vulneraveis", "qtd_pes_pob_vul","qtd_0a11"]]

## Hora de plotar!!

vamos analisar como cada grupo esta relacionado com a pobreza

In [62]:
dftemp = newDataframe

dftemp = dftemp[(dftemp['local']== 0)]


mapeamento = {
    1: 'Homem Branco',
    2: 'Mulher Branca',
    3: 'Homem Negro',
    4: 'Mulher Negra'
}

dftemp['sxraca'] = dftemp['sxraca'].map(mapeamento)


fig = sp.make_subplots(rows=2, cols=2)

fig1 = px.bar(dftemp, x='sxraca', y='qtd_pessoas')
fig.add_trace(fig1['data'][0], row=1, col=1)

fig2 = px.bar(dftemp, x='sxraca', y='qtd_pes_pobres')
fig.add_trace(fig2['data'][0], row=1, col=2)

fig3 = px.bar(dftemp, x='sxraca', y='qtd_pes_vulneraveis')
fig.add_trace(fig3['data'][0], row=2, col=1)

fig4 = px.bar(dftemp, x='sxraca', y='qtd_pes_pob_vul')
fig.add_trace(fig4['data'][0], row=2, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas', row=1, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres', row=1, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas Vulneráveis', row=2, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres ou vulneráveis', row=2, col=2)
fig.update_layout(height=1000)
fig.show()


/var/folders/qr/gzc51r8560v_3fjtkyl6bdvc0000gn/T/ipykernel_92206/4271452727.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Com isso conseguimos analizar nossos dados ainda estao incosistentes...
    Repare que nossa base de dados tem mais mulheres brancas em situação de pob ou vul do que existe de mulheres brancas.
    Para resolver isso vamos deduzir que: 

    qtd_pes_pob_vul = qtd_pes_pobres + qtd_pes_vulneraveis

In [63]:
dftemp['qtd_pes_pob_vul'] = dftemp["qtd_pes_pobres"] + dftemp["qtd_pes_vulneraveis"]

/var/folders/qr/gzc51r8560v_3fjtkyl6bdvc0000gn/T/ipykernel_92206/2042387282.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [64]:
fig = sp.make_subplots(rows=2, cols=2)

fig1 = px.bar(dftemp, x='sxraca', y='qtd_pessoas')
fig.add_trace(fig1['data'][0], row=1, col=1)

fig2 = px.bar(dftemp, x='sxraca', y='qtd_pes_pobres')
fig.add_trace(fig2['data'][0], row=1, col=2)

fig3 = px.bar(dftemp, x='sxraca', y='qtd_pes_vulneraveis')
fig.add_trace(fig3['data'][0], row=2, col=1)

fig4 = px.bar(dftemp, x='sxraca', y='qtd_pes_pob_vul')
fig.add_trace(fig4['data'][0], row=2, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas', row=1, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres', row=1, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas Vulneráveis', row=2, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres ou vulneráveis', row=2, col=2)
fig.update_layout(height=1000)
fig.show()

### Com esses graficos conseguimos analisar que: 

    O grupo de homens brancos é o menos suscetível a estar em situação de vulnerabilidade e pobreza, enquanto o homem negro é o mais presente nesse grupo

    Entre brancos existem mais mulheres em situação de vulnerabilidade ou pobreza, enquanto entre negros os homens predominam

    Homens brancos praticamente nao sofrem com a pobreza, a mulher branca aparece com mais frequência em situação de pobreza

    A raca parece ser um fator de importância ao se estudar pessoas em situação de vulnerabilidade ou pobreza, entretanto o gênero nao parece ser um fator de grande relevância

## Vamos estudar mais de perto como o gênero e raca se relacionam com pobreza e vulnerabilidade para tirar nossas conclusões

In [65]:


dfGenero = dftemp.copy()
mapeamento = {
     'Homem Branco' :"Homem",
     'Mulher Branca' :"Mulher",
     'Homem Negro' :"Homem",
     'Mulher Negra' :"Mulher"
}

dfGenero['sxraca'] = dfGenero['sxraca'].replace(mapeamento)


dfGenero = dfGenero.groupby('sxraca').sum().reset_index()

fig = sp.make_subplots(rows=2, cols=2)

fig1 = px.bar(dfGenero, x='sxraca', y='qtd_pessoas')
fig.add_trace(fig1['data'][0], row=1, col=1)

fig2 = px.bar(dfGenero, x='sxraca', y='qtd_pes_pobres')
fig.add_trace(fig2['data'][0], row=1, col=2)

fig3 = px.bar(dfGenero, x='sxraca', y='qtd_pes_vulneraveis')
fig.add_trace(fig3['data'][0], row=2, col=1)

fig4 = px.bar(dfGenero, x='sxraca', y='qtd_pes_pob_vul')
fig.add_trace(fig4['data'][0], row=2, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas', row=1, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres', row=1, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas Vulneráveis', row=2, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres ou vulneráveis', row=2, col=2)
fig.update_layout(height=1000)
fig.show()


    A análise estatística revela que, apesar dos homens serem mais afetados por pobreza e vulnerabilidade, há uma distribuição semelhante entre os gêneros. Isso sugere que a pobreza e a vulnerabilidade não discriminam com base no gênero, afetando homens e mulheres de maneira quase igual.

    Agora vamos estudar mais sobre a raca...

In [66]:


dfRaca = dftemp.copy()
mapeamento = {
     'Homem Branco' :"Branco",
     'Mulher Branca' :"Branco",
     'Homem Negro' :"Negro",
     'Mulher Negra' :"Negro"
}

dfRaca['sxraca'] = dfRaca['sxraca'].replace(mapeamento)


dfRaca = dfRaca.groupby('sxraca').sum().reset_index()

fig = sp.make_subplots(rows=2, cols=2)

fig1 = px.bar(dfRaca, x='sxraca', y='qtd_pessoas')
fig.add_trace(fig1['data'][0], row=1, col=1)

fig2 = px.bar(dfRaca, x='sxraca', y='qtd_pes_pobres')
fig.add_trace(fig2['data'][0], row=1, col=2)

fig3 = px.bar(dfRaca, x='sxraca', y='qtd_pes_vulneraveis')
fig.add_trace(fig3['data'][0], row=2, col=1)

fig4 = px.bar(dfRaca, x='sxraca', y='qtd_pes_pob_vul')
fig.add_trace(fig4['data'][0], row=2, col=2)

fig.update_yaxes(title_text='Quantidade de Pessoas', row=1, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres', row=1, col=2)
fig.update_yaxes(title_text='Quantidade de Pessoas Vulneráveis', row=2, col=1)
fig.update_yaxes(title_text='Quantidade de Pessoas Pobres ou vulneráveis', row=2, col=2)

fig.update_layout(height=1000)
fig.show()


    A análise estatística indica que, embora a pobreza e a vulnerabilidade afetem todos os grupos demográficos, os negros são exorbitantemente mais afetados do que os brancos. Isso sugere que a pobreza e a vulnerabilidade, infelizmente, discriminam com base na raça, impactando desproporcionalmente a população negra.




    

### Agora vamos estudar o comportamento por região!

    Sabendo da inconsistência em muitas linhas da base escolhida, vamos usar agora uma outra planilha para analisar a distribuição de pobreza por região brasileira

In [67]:
dataframeBase = pd.read_excel("./Agregações GR UF Brasil/Grandes Regiões.ods",engine='odf')

newDataframe = dataframeBase[["cod_local", "qtd_pes", "qtd_pes_pobres", "qtd_pes_vulneraveis", "qtd_pes_pob_vul","qtd_0a11"]].copy()

mapeamento = {
1: "Norte",
2: "Nordeste",
3: "Sudeste",
4: "Sul",
5: "Centro-Oeste"
}
newDataframe['cod_local'] = newDataframe['cod_local'].replace(mapeamento)



fig = sp.make_subplots(rows=1, cols=2)

fig1 = px.bar(newDataframe, x = 'cod_local', y = 'qtd_pes')
fig.add_trace(fig1['data'][0], row=1, col=1)

fig1 = px.bar(newDataframe, x = 'cod_local', y = 'qtd_pes_pob_vul')
fig.add_trace(fig1['data'][0], row=1, col=2)


fig.update_yaxes(title_text='Quantidade de Pessoas',row=1, col=1,)
fig.update_yaxes(title_text='Quantidade de Pessoas pobres ou vulneráveis',row=1, col=2,)

fig.show()


In [69]:
dataframeBase = pd.read_excel("Agregações GR UF Brasil/UF.ods",engine='odf')

newDataframe = dataframeBase[["cod_local", "qtd_pes", "qtd_pes_pobres", "qtd_pes_vulneraveis", "qtd_pes_pob_vul","qtd_0a11"]].copy()

mapeamento = {
11: "Rondônia",
12: "Acre",
13: "Amazonas",
14: "Roraima",
15: "Pará",
16: "Amapá",
17: "Tocantins",
21: "Maranhão",
22: "Piauí",
23: "Ceará",
24: "Rio Grande do Norte",
25: "Paraíba",
26: "Pernambuco",
27: "Alagoas",
28: "Sergipe",
29: "Bahia",
31: "Minas Gerais",
32: "Espírito Santo",
33: "Rio de Janeiro",
35: "São Paulo",
41: "Paraná",
42: "Santa Catarina",
43: "Rio Grande do Sul",
50: "Mato Grosso do Sul",
51: "Mato Grosso",
52: "Goiás",
53: "Distrito Federal",
}

newDataframe['cod_local'] = newDataframe['cod_local'].replace(mapeamento)


fig1 = px.bar(newDataframe, x = 'cod_local', y = 'qtd_pes')
fig1.update_yaxes(title_text='Quantidade de Pessoas')
fig1.update_xaxes(title_text='Estado')

fig1.show()


fig1 = px.bar(newDataframe, x = 'cod_local', y = 'qtd_pes_pob_vul')
fig1.update_yaxes(title_text='Quantidade de Pessoas pobres ou vulneráveis')
fig1.update_xaxes(title_text='Estado')


fig1.show()


In [86]:

dataframeBase = pd.read_excel("MunicipioBrasil_20230102.xlsx",)

newDataframe = dataframeBase[["nom_mun", "qtd_pes_pob_vul"]].copy()

top_10 = newDataframe.nlargest(20,'qtd_pes_pob_vul')

fig1 = px.bar(top_10, x = 'nom_mun', y = 'qtd_pes_pob_vul', title="Top 20 Municipios com maior quantidade de pessoas pobres ou vulneráveis ")
fig1.update_yaxes(title_text='Quantidade de Pessoas pobres ou vulneráveis')
fig1.update_xaxes(title_text='Municipio')
fig1.show()

In [94]:

dataframeBase['proporcao'] = (dataframeBase['qtd_pes_pob_vul'] / dataframeBase['qtd_pes']) * 100


top_20_cidades_df = dataframeBase.nlargest(20, 'proporcao').round(2)

fig1 = px.bar(top_20_cidades_df, x = 'nom_mun', y = 'proporcao', title="Top 20 Municipios com maior quantidade de pessoas pobres ou vulneráveis")
fig1.update_yaxes(title_text='% de Pessoas pobres ou vulneráveis')
fig1.update_xaxes(title_text='Municipio')

for i, v in enumerate(top_20_cidades_df['proporcao']):
    fig1.add_annotation(dict(x=top_20_cidades_df['nom_mun'].iloc[i], y=v,
                             text=str(v),
                             showarrow=True,
                             font=dict(size=10)
                            )
                       )
fig1.show()

In [96]:
fig = px.choropleth(dataframeBase, geojson=brasil, locations='cod_mun', color='qtd_pes_pob_vul',
                           color_continuous_scale="Viridis",
                           labels={'qtd_pes_pob_vul':'Quantidade de Pessoas em Situação de Vulnerabilidade'}
                          )

fig.update_geos(showcountries=False, showcoastlines=True, showland=True, fitbounds="locations")

fig.show()